# Descarga de resultados de elecciones generales disponibles en el Atlas Electoral del OEP

El Órgano Electoral mantiene un [atlas con datos de resultados electorales](https://atlaselectoral.oep.org.bo/). En este notebook descargo todos los datos disponibles para elecciones generales.

In [1]:
import requests
import pandas as pd
import os
from IPython.display import Markdown, display

Creamos una lista de los procesos electorales disponibles en el Atlas Electoral.

In [2]:
def listar_procesos():
    def parse_categoria(categoria):
        def parse_level(level, sublevel_key):
            return [
                {k: level[k] for k in level.keys() if k != sublevel_key},
                level[sublevel_key],
            ]

        data = []
        data_categoria, procesos = parse_level(categoria, "proceso_electorals")
        for proceso in procesos:
            data_proceso, subprocesos = parse_level(proceso, "sub_procesos")
            for subproceso in subprocesos:
                data.append({**data_categoria, **data_proceso, **subproceso})
        return data

    response = requests.get(
        "https://atlaselectoral.oep.org.bo/api/proceso-electoral-menu"
    )
    resultado = []
    for categoria in response.json():
        resultado.extend(parse_categoria(categoria))
    return pd.DataFrame(resultado)

In [3]:
procesos = listar_procesos()

In [4]:
Markdown(
    f"Existen {len(procesos.id_agrupacion_proceso.unique())} categorías con {len(procesos.id_proceso_electoral.unique())} procesos y {len(procesos.id_sub_proceso.unique())} subprocesos electorales."
)

Existen 7 categorías con 48 procesos y 130 subprocesos electorales.

Éstos son los subprocesos en la categoría "_Elecciones Municipales_":

In [5]:
procesos[procesos.nom_agrupacion_proceso == "Elecciones Municipales"][
    ["nombre_proceso_electoral", "nom_sub_proceso", "id_sub_proceso"]
]

,nombre_proceso_electoral,nom_sub_proceso,id_sub_proceso
51,Elecciones Municipales 1985,Elecciones Municipales 1985,63
52,Elecciones Municipales 1987,Elecciones Municipales 1987,71
53,Elecciones Municipales 1989,Elecciones Municipales 1989,72
54,Elecciones Municipales 1991,Elecciones Municipales 1991,64
55,Elecciones Municipales 1993,Elecciones Municipales 1993,65
56,Elecciones Municipales 1995,Elecciones Municipales 1995,66
57,Elecciones Municipales 1999,Elecciones Municipales 1999,1
58,Elecciones Municipales 2004,Elecciones Municipales 2004,67
59,Elecciones Municipales 2010,Elección de Alcaldes 2010,25
60,Elecciones Municipales 2010,Elección de Concejales 2010,26


Intentamos descargar resultados para estos procesos.

In [6]:
def descargar_resultados(subproceso):
    headers = {
        "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:141.0) Gecko/20100101 Firefox/141.0",
    }

    columnas_admin = {
        "Codigo PAIS": "pais",
        "Nombre PAIS": "pais_nombre",
        "Codigo DEPARTAMENTO": "departamento",
        "Nombre DEPARTAMENTO": "departamento_nombre",
        "Codigo PROVINCIA": "provincia",
        "Nombre PROVINCIA": "provincia_nombre",
        "Codigo MUNICIPIO": "municipio",
        "Nombre MUNICIPIO": "municipio_nombre",
        "Codigo CIRCUNSCRIPCION": "circunscripcion",
        "Nombre CIRCUNSCRIPCION": "circunscripcion_nombre",
        "Codigo RECINTO": "recinto",
        "Nombre RECINTO": "recinto_nombre",
        "Codigo MESA": "mesa",
        "Nombre MESA": "mesa_nombre",
    }
    directorio = subproceso["nombre_proceso_electoral"].split()[-1]
    filename = subproceso["nom_sub_proceso"].lower().replace(" ", "_")
    url = f"https://atlaselectoral.oep.org.bo/descarga/{subproceso['id_sub_proceso']}/votos_totales.csv"
    os.makedirs(directorio, exist_ok=True)

    try:
        df = pd.read_csv(url, sep="|", storage_options=headers)
        admin = [col for col in df.columns if col in columnas_admin.keys()]
        stacked = df.set_index(admin).stack().reset_index().copy()
        stacked.columns = [columnas_admin[col] for col in admin] + ["opcion", "conteo"]
        stacked.to_parquet(f"{directorio}/{filename}.parquet")
        display(
            Markdown(
                f"**{subproceso['nom_sub_proceso']}**: desagregado en {', '.join(['`' + col.split()[-1].lower() + '`' for col in admin if 'Codigo' in col])}"
            )
        )
    except Exception as e:
        display(Markdown(f"{subproceso['nom_sub_proceso']}: ERROR {e}"))

In [7]:
for i, subproceso in procesos[
    procesos.nom_agrupacion_proceso == "Elecciones Municipales"
].iterrows():
    descargar_resultados(subproceso)

**Elecciones Municipales 1985**: desagregado en `pais`, `departamento`

**Elecciones Municipales 1987**: desagregado en `pais`, `departamento`, `provincia`, `municipio`

**Elecciones Municipales 1989**: desagregado en `pais`, `departamento`, `provincia`

**Elecciones Municipales 1991**: desagregado en `pais`, `departamento`, `provincia`, `municipio`

**Elecciones Municipales 1993**: desagregado en `pais`, `departamento`, `provincia`

...

**Elección de Alcalde en Chuquihuta 2018**: desagregado en `pais`, `departamento`, `provincia`, `municipio`, `recinto`, `mesa`

**Elección de Alcalde en Cotagaita 2018**: desagregado en `pais`, `departamento`, `provincia`, `municipio`, `recinto`, `mesa`

**ELECCIÓN DE ALCALDES 2021**: desagregado en `pais`, `departamento`, `provincia`, `municipio`, `recinto`, `mesa`

**ELECCIÓN DE CONCEJALES 2021**: desagregado en `pais`, `departamento`, `provincia`, `municipio`, `recinto`, `mesa`

**ELECCIÓN EXTRAORDINARIA DE ALCALDE DE LOS GOBIERNOS AUTÓNOMOS MUNICIPALES DE LA GUARDIA Y SAN JAVIER 2022**: desagregado en `pais`, `departamento`, `provincia`, `municipio`, `recinto`, `mesa`